In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
data = pd.read_csv(".\properatti.csv", index_col=0)
data.shape

In [ ]:
# grupos de columas para poder trabajar con ellas de forma mas sencilla
geolocation = ['state_name','geonames_id', 'lat-lon','lat','lon']

# Creamos un nuevo dataframe con las columnas que me interesan

geo_location_data = (data[ geolocation].copy())
geo_location_data


In [ ]:
masc_nulos_lat = geo_location_data["lat"].astype(str)  == "nan"
geo_location_data[masc_nulos_lat].index

In [ ]:
masc_nulos_lon = geo_location_data["lon"].astype(str)  == "nan"
geo_location_data[masc_nulos_lon].index

In [ ]:
#como lat y long tienen las mismas filas nan, escojo uno para obtener el indice
filas_nan= geo_location_data[masc_nulos_lon].index
print(geo_location_data.loc[filas_nan, ["state_name",'geonames_id','lat','lon']])


In [ ]:
df_geo_nulos = geo_location_data.loc[filas_nan, ["state_name",'geonames_id','lat','lon']]
#para tener una idea donde están concentrados los datos sin lat-lon 
df_geo_nulos["state_name"].value_counts().head(10) ##

In [ ]:
#saco los nulos de geonames_id
df_geo_nulos.dropna(subset=['geonames_id'], inplace=True)
df_geo_nulos.isnull().sum()

#y lat y lon nuevas que tengo que buscar para completar para completar

Trabajando con la data externa de geonames 

In [ ]:
geonames = pd.read_csv("ar_copy.csv", sep='\t', header=None)
#como no tengo nombres en la columnas para mejorar la extraccion renombro las que me interesan
geonames.rename({0: 'geoname_oficial', 4:"lat_oficial", 5:"lon_oficial"}, axis=1, inplace=True)
geonames.head(4)

In [ ]:
#vemos que geoname_oficial es tipo int mientras geoname_id es float, por lo cual lo cambiamos
df_geo_nulos["geonames_id"] = df_geo_nulos["geonames_id"].astype(int)

In [ ]:
#tambien vamos a sacar los nulos del geoname_oficial y por 
geonames.dropna(subset=['geoname_oficial'], inplace=True)
geonames[["geoname_oficial","lat_oficial","lon_oficial"]].isnull().sum().head(6)

In [ ]:
#ahora si vamos a buscar los datos de lat y lon desde el geoname_oficial del archivo geonames de internet
#Creamos un diccionario vacio para ubicar la Latitud
lat_dict = {}

#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lat = zip(geonames['geoname_oficial'], geonames['lat_oficial'])

#Rellenamos el diccionario
for geoname, lat_oficial in geoname_lat:
    lat_dict[geoname] = lat_oficial

In [ ]:
#Creamos un diccionario para ubicar la Longitud
lon_dict = {}

#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lon = zip(geonames['geoname_oficial'], geonames['lon_oficial'])

#Rellenamos el diccionario
for geoname, lon_oficial in geoname_lon:
    lon_dict[geoname] = lon_oficial

In [ ]:
df_geo_nulos["lat"] = df_geo_nulos['geonames_id'].map(lat_dict) 
df_geo_nulos["lon"] = df_geo_nulos['geonames_id'].map(lon_dict)

In [ ]:
#cambio el nombre del data completado
df_geo_completo = df_geo_nulos
df_geo_completo

In [ ]:
geo_location_data.isna().sum()

In [ ]:
df_geo_completo.isna().sum()

In [ ]:
porcentaje_completado_lat_lon = (df_geo_completo[["lat","lon"]].isna().sum() / geo_location_data[["lat","lon"]].isnull().sum())*100
porcentaje_completado_lat_lon